# Install Required Libraries
Install the necessary Python libraries and load settings.

In [ ]:
%pip install azure-ai-contentsafety
%pip install dotenv

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv("credentials.env")
api_key=os.getenv('AZURE_CONTENT_SAFETY_KEY')
endpoint=os.getenv('AZURE_CONTENT_SAFETY_ENDPOINT')

print(endpoint)

# Defining the Image Moderation Function
Scan images and detect potential adult and racy content by using tags, confidence scores, and other extracted information. Code to define a function that takes an image URL as input and calls the Azure Content Safety API for moderation:

In [ ]:
import os

from azure.ai.contentsafety import ContentSafetyClient
from azure.ai.contentsafety.models import AnalyzeImageOptions, ImageData, ImageCategory
from azure.core.credentials import AzureKeyCredential
from azure.core.exceptions import HttpResponseError

def analyze_image():
    endpoint = os.environ.get('AZURE_CONTENT_SAFETY_ENDPOINT')
    key = os.environ.get('AZURE_CONTENT_SAFETY_KEY')
    image_path = os.path.join("sample", "knife.jpg")

    # Create an Azure AI Content Safety client
    client = ContentSafetyClient(endpoint, AzureKeyCredential(key))


    # Build request
    with open(image_path, "rb") as file:
        request = AnalyzeImageOptions(image=ImageData(content=file.read()))

    # Analyze image
    try:
        response = client.analyze_image(request)
    except HttpResponseError as e:
        print("Analyze image failed.")
        if e.error:
            print(f"Error code: {e.error.code}")
            print(f"Error message: {e.error.message}")
            raise
        print(e)
        raise

    hate_result = next(item for item in response.categories_analysis if item.category == ImageCategory.HATE)
    self_harm_result = next(item for item in response.categories_analysis if item.category == ImageCategory.SELF_HARM)
    sexual_result = next(item for item in response.categories_analysis if item.category == ImageCategory.SEXUAL)
    violence_result = next(item for item in response.categories_analysis if item.category == ImageCategory.VIOLENCE)

    if hate_result:
        print(f"Hate severity: {hate_result.severity}")
    if self_harm_result:
        print(f"SelfHarm severity: {self_harm_result.severity}")
    if sexual_result:
        print(f"Sexual severity: {sexual_result.severity}")
    if violence_result:
        print(f"Violence severity: {violence_result.severity}")

if __name__ == "__main__":
    analyze_image()

# Defining the Text Moderation Function
Scan text content. Profanity terms and personal data are returned. Code to define a function that takes text as input and calls the Azure Content Safety API for moderation:

In [ ]:
# coding: utf-8

# -------------------------------------------------------------------------
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT License. See License.txt in the project root for
# license information.
# --------------------------------------------------------------------------
import os
from azure.ai.contentsafety import ContentSafetyClient
from azure.core.credentials import AzureKeyCredential
from azure.core.exceptions import HttpResponseError
from azure.ai.contentsafety.models import AnalyzeTextOptions, TextCategory


# Sample: Analyze text in sync request
def analyze_text():
    # analyze text
    key = os.environ["AZURE_CONTENT_SAFETY_KEY"]
    endpoint = os.environ["AZURE_CONTENT_SAFETY_ENDPOINT"]

    # Create a Content Safety client
    client = ContentSafetyClient(endpoint, AzureKeyCredential(key))

    # Construct request
    request = AnalyzeTextOptions(text="You are an idiot")

    # Analyze text
    try:
        response = client.analyze_text(request)
    except HttpResponseError as e:
        print("Analyze text failed.")
        if e.error:
            print(f"Error code: {e.error.code}")
            print(f"Error message: {e.error.message}")
            raise
        print(e)
        raise

    hate_result = next(item for item in response.categories_analysis if item.category == TextCategory.HATE)
    self_harm_result = next(item for item in response.categories_analysis if item.category == TextCategory.SELF_HARM)
    sexual_result = next(item for item in response.categories_analysis if item.category == TextCategory.SEXUAL)
    violence_result = next(item for item in response.categories_analysis if item.category == TextCategory.VIOLENCE)

    if hate_result:
        print(f"Hate severity: {hate_result.severity}")
    if self_harm_result:
        print(f"SelfHarm severity: {self_harm_result.severity}")
    if sexual_result:
        print(f"Sexual severity: {sexual_result.severity}")
    if violence_result:
        print(f"Violence severity: {violence_result.severity}")


if __name__ == "__main__":
    analyze_text()

In [ ]:
#install video dependencies
%pip install decord

# Defining the Video Moderation Function
Scan videos and detect potential adult and racy content. Code to define a function that takes video as input and calls the Azure Content Safety API for moderation:

In [ ]:
# coding: utf-8

# -------------------------------------------------------------------------
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT License. See License.txt in the project root for
# license information.
# --------------------------------------------------------------------------
import os

from azure.ai.contentsafety import ContentSafetyClient
from azure.ai.contentsafety.models import AnalyzeImageOptions, ImageData, ImageCategory
from azure.core.credentials import AzureKeyCredential
from azure.core.exceptions import HttpResponseError

import datetime
from decord import VideoReader
from io import BytesIO
import numpy as np
from tqdm import tqdm
from PIL import Image


# Sample: Analyze video in sync request
def analyze_video():
    # [START analyze_video]
    key = os.environ["AZURE_CONTENT_SAFETY_KEY"]
    endpoint = os.environ["AZURE_CONTENT_SAFETY_ENDPOINT"]
    video_path = os.path.join("sample", "video.mp4")
    # Create a Content Safety client
    client = ContentSafetyClient(endpoint, AzureKeyCredential(key))

    # Read the video
    video = VideoReader(video_path)
    sampling_fps = 1  # process at 1 frames per second
    key_frames = video.get_batch(
        np.int_(np.arange(0, len(video), video.get_avg_fps() / sampling_fps))).asnumpy()

    # Iterate over key frames
    for key_frame_idx in tqdm(range(len(key_frames)), desc="Processing video",
                              total=len(key_frames)):
        frame = Image.fromarray(key_frames[key_frame_idx])
        frame_bytes = BytesIO()
        frame.save(frame_bytes, format="PNG")

        # Build request
        request = AnalyzeImageOptions(image=ImageData(content=frame_bytes.getvalue()))

        # Analyze image
        frame_time_ms = key_frame_idx * 1000 / sampling_fps
        frame_timestamp = datetime.timedelta(milliseconds=frame_time_ms)
        print(f"Analyzing video at {frame_timestamp}")
        try:
            response = client.analyze_image(request)
        except HttpResponseError as e:
            print(f"Analyze video failed at {frame_timestamp}")
            if e.error:
                print(f"Error code: {e.error.code}")
                print(f"Error message: {e.error.message}")
                raise
            print(e)
            raise

        hate_result = next(
            item for item in response.categories_analysis if item.category == ImageCategory.HATE)
        self_harm_result = next(item for item in response.categories_analysis if
                                item.category == ImageCategory.SELF_HARM)
        sexual_result = next(
            item for item in response.categories_analysis if item.category == ImageCategory.SEXUAL)
        violence_result = next(item for item in response.categories_analysis if
                               item.category == ImageCategory.VIOLENCE)

        if hate_result:
            print(f"Hate severity: {hate_result.severity}")
        if self_harm_result:
            print(f"SelfHarm severity: {self_harm_result.severity}")
        if sexual_result:
            print(f"Sexual severity: {sexual_result.severity}")
        if violence_result:
            print(f"Violence severity: {violence_result.severity}")

    # [END analyze_video]


if __name__ == "__main__":
    analyze_video()

[mslearn link](https://learn.microsoft.com/en-us/azure/ai-services/content-moderator/api-reference)